In [1]:
from pyueye import ueye
import numpy as np
import cv2
import sys
import time
import datetime
from PIL import Image
import os
import matplotlib.pyplot as plt
from skimage.io import imsave
k=300

In [2]:
gain_arr=np.arange(120,420,40)[::-1]
b=gain_arr
for i in range(len(gain_arr)-1):
    gain_arr=np.append(gain_arr,b)
    
exposure_arr=np.arange(1,33,4)
exposure_arr=np.repeat(exposure_arr,len(exposure_arr))
print(gain_arr)
print(exposure_arr)

[400 360 320 280 240 200 160 120 400 360 320 280 240 200 160 120 400 360
 320 280 240 200 160 120 400 360 320 280 240 200 160 120 400 360 320 280
 240 200 160 120 400 360 320 280 240 200 160 120 400 360 320 280 240 200
 160 120 400 360 320 280 240 200 160 120]
[ 1  1  1  1  1  1  1  1  5  5  5  5  5  5  5  5  9  9  9  9  9  9  9  9
 13 13 13 13 13 13 13 13 17 17 17 17 17 17 17 17 21 21 21 21 21 21 21 21
 25 25 25 25 25 25 25 25 29 29 29 29 29 29 29 29]


In [3]:
def capture_pic(exposure_arr,gain_arr,k): 
    def get_bits_per_pixel(color_mode):
        formats = {ueye.IS_CM_SENSOR_RAW8: 8,
                   ueye.IS_CM_SENSOR_RAW10: 16,
                   ueye.IS_CM_SENSOR_RAW12: 16,
                   ueye.IS_CM_SENSOR_RAW16: 16,

                   ueye.IS_CM_MONO8: 8,
                   ueye.IS_CM_MONO10: 16,
                   ueye.IS_CM_MONO12: 16,
                   ueye.IS_CM_MONO16: 16,

                   ueye.IS_CM_RGB8_PACKED: 24,
                   ueye.IS_CM_RGBA8_PACKED: 32,
                   ueye.IS_CM_RGBY8_PACKED: 32,
                   ueye.IS_CM_RGB10_PACKED: 32,

                   ueye.IS_CM_RGB10_UNPACKED: 48,
                   ueye.IS_CM_RGB12_UNPACKED: 48,
                   ueye.IS_CM_RGBA12_UNPACKED: 64,

                   ueye.IS_CM_BGR5_PACKED: 16,
                   ueye.IS_CM_BGR565_PACKED: 16,
                   ueye.IS_CM_BGR8_PACKED: 24,
                   ueye.IS_CM_BGRA8_PACKED: 32,
                   ueye.IS_CM_BGRY8_PACKED: 32,
                   ueye.IS_CM_BGR10_PACKED: 32,

                   ueye.IS_CM_BGR10_UNPACKED: 48,
                   ueye.IS_CM_BGR12_UNPACKED: 48,
                   ueye.IS_CM_BGRA12_UNPACKED: 64,

                   ueye.IS_CM_UYVY_PACKED: 16,
                   ueye.IS_CM_UYVY_MONO_PACKED: 16,
                   ueye.IS_CM_UYVY_BAYER_PACKED: 16,
                   ueye.IS_CM_CBYCRY_PACKED: 16
                   }

        return formats[color_mode]

    hCam = ueye.HIDS(0)                                                                          # Kamera Handler
    CamStat_Init = ueye.is_InitCamera(hCam, None)                                                # Init Kamera, None = der DIB-Modus wird zur Bildausgabe verwendet
    """Return Values:
    IS_NO_SUCCESS = -1
    IS_SUCCESS = 0
    IS_INVALID_CAMERA_HANDLE = 1
    IS_INVALID_HANDLE = 1
    IS_IO_REQUEST_FAILED = 2
    IS_CANT_OPEN_DEVICE = 3 ; ......"""


    #################### Change Inintial Camera Settings ####################

    ueye.is_SetSubSampling (hCam, ueye.IS_SUBSAMPLING_2X_VERTICAL | ueye.IS_SUBSAMPLING_2X_HORIZONTAL) # subsampling aktivieren
    #ueye.is_SetSubSampling (hCam, ueye.IS_SUBSAMPLING_DISABLE)
    CamProp_FPS = ueye.double()
    ueye.is_SetFrameRate (hCam, 15, CamProp_FPS)                                                    # Framerate einstellen
    
    CamProp_Exposure = ueye.double()
    #CamProp_Exposure.value = exposure_arr[0]     # exposure value in ms ; max=32 if FPS=30
    #expo=exposure_arr[0]
    #nRet = ueye.is_Exposure (hCam, ueye.IS_EXPOSURE_CMD_SET_EXPOSURE, CamProp_Exposure, 8)                        # Belichtungszeit einstellen

    CamProp_ExposureReal = ueye.double()
    #nRet = ueye.is_Exposure (hCam, ueye.IS_EXPOSURE_CMD_GET_EXPOSURE, CamProp_ExposureReal, 8)     

    CamProp_Gamma = ueye.int()
    #CamProp_Gamma.value = gain_arr[0]
    #gain=gain_arr[0]
   #ueye.is_Gamma(hCam, ueye.IS_GAMMA_CMD_SET, CamProp_Gamma, ueye.sizeof(CamProp_Gamma)) 

   
    #########################################################


    aoi = ueye.IS_RECT()                                                                            #
    pcImageMem = ueye.c_mem_p()                                                                     #
    pid = ueye.int()     


                                                                             #
    bitspixel = get_bits_per_pixel(ueye.is_SetColorMode(hCam, ueye.IS_GET_COLOR_MODE))              #
    bytes_per_pixel = int(bitspixel / 8)



    ueye.is_AOI(hCam, ueye.IS_AOI_IMAGE_GET_AOI, aoi, ueye.sizeof(aoi))                             # Alloc
    width = ueye.INT(aoi.s32Width)                                                                  #
    height = ueye.INT(aoi.s32Height)                                                                #



    #### Ringspeicher anlegen
    for i in range (0,100):
        ueye.is_AllocImageMem(hCam, width, height, bitspixel, pcImageMem, pid)                          # Speicher anlegen
        msg= ueye.is_AddToSequence(hCam, pcImageMem, pid)


    #ueye.is_SetImageMem(hCam, pcImageMem, pid)                                                      # Speicher der Kamera zuweisen
    ueye.is_InitImageQueue(hCam, nMode=0) # Warteschlangenmodus aktivieren 




    #ueye.is_CaptureVideo (hCam, Wait = 50)
    ueye.is_CaptureVideo (hCam, ueye.IS_WAIT)



    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    now = datetime.datetime.now()
    now_format = now.strftime("%Y-%m-%d_%H-%M-%S")
   # out_name = 'C:/Users/sinayak/Project/Video' + str(now_format) +\
   # '_IDS_Cam_FPS_' + str("%.1f" % CamProp_FPS.value)+ '_Exp_' + str("%.1f" % CamProp_ExposureReal.value) + str('.avi')



    #out = cv2.VideoWriter('output.avi',fourcc, CamProp_FPS.value, (width,height))
    #out = cv2.VideoWriter(out_name,0 , CamProp_FPS.value, (height,width)) #Output path, codec, fps, resolution

    time_start = time.perf_counter()

    j=0
    i=0
    while True:
        #prevExpo=expo
        #prevGain=gain
        ueye.is_WaitForNextImage (hCam, 1000, pcImageMem, pid)
        expo=exposure_arr[j] 
        CamProp_Exposure.value = exposure_arr[j]     # change the exposure value in ms ; max=32 if FPS=30
        nRet = ueye.is_Exposure (hCam, ueye.IS_EXPOSURE_CMD_SET_EXPOSURE, CamProp_Exposure, 8)  
        CamProp_Gamma.value = gain_arr[j]                     # change the gains 
        gain=gain_arr[j]
        ueye.is_Gamma(hCam, ueye.IS_GAMMA_CMD_SET, CamProp_Gamma, ueye.sizeof(CamProp_Gamma)) 

        


        #if (ueye.is_CaptureVideo (hCam, ueye.IS_GET_LIVE) == ueye.IS_SUCCESS):is_WaitForNextImage (HIDS hCam, UINT timeout, char** ppcMem, INT* imageID)
        pcDest = (ueye.CHAR * int(((width * height * bitspixel)/8)))()                # pcImageMem(LP_c_char) copied to
        ueye.is_CopyImageMem(hCam=hCam, pcSource=pcImageMem, nID=pid, pcDest=pcDest)  # pcDest(c_char_Array_x)

        pcDest = np.reshape(pcDest, (int(height), int(width), bytes_per_pixel))  # Restructuring pcDest to numpy (*1)
        pcDest = pcDest.view(np.uint8)                                           # Data viewed as uint8


        #cv2.imshow("pcDest", pcDest)
        #cv2.imshow("mean", real_mean_int)
        
        if i<=6:
            print(i)
            i+=1
            continue
        
        print(j)
        
        
        #### fps bestimmen
        CamProp_RealFps = ueye.double()
        ueye.is_GetFramesPerSecond (hCam, CamProp_RealFps)
        print("FPS: " + str(CamProp_RealFps.value))

        CamProp_RealExp = ueye.double()
        ueye.is_Exposure (hCam, ueye.IS_EXPOSURE_CMD_GET_EXPOSURE, CamProp_RealExp, 8)
        print("Exposure: " + str(CamProp_RealExp.value))

        # write the image to the file
        #out.write(pcDest)
        result=Image.fromarray((pcDest*255).astype(np.uint8))
        newpath=r'C:/Users/sinayak/Project/Video/test'+str(k)+'/'
        if not os.path.exists(newpath):
            os.makedirs(newpath)
        #result.save(newpath+str('/')+str(j)+'.jpg')
        #matplotlib.image.imsave(newpath+str('/')+str(j)+'.jpg',pcDest)
        #plt.imshow(pcDest)
        #plt.savefig(newpath+str('/')+str(j)+'.jpg')
        #imsave(newpath+str('/')+str(j)+'.jpg',pcDest)
        cv2.imwrite(newpath+str(float("{0:.2f}".format(expo)))+'_'+str(gain)+'.jpg',pcDest,[int(cv2.IMWRITE_JPEG_QUALITY),100])
        #cv2.cvtColor(pcDest,cv2.COLOR_BGR2RGB)

        time_end = time.perf_counter()
        i = i+1
        print(str((time_end-time_start)))
        time_start = time.perf_counter()
        #cv2.imwrite('trash/image'+str(i)+'.jpg', pcDest)  

        ueye.is_UnlockSeqBuf (hCam, ueye.IS_IGNORE_PARAMETER, pcImageMem)

        j+=1
        i+=1
        key = cv2.waitKey(20)
        if key == 27: # exit on ESC
            break
        if j==len(exposure_arr)*len(gain_arr)-1:
            break

    #cv2.imwrite('image.jpg', pcDest)                                             # Save as image
    cv2.destroyAllWindows()


    ueye.is_FreeImageMem(hCam, pcImageMem, pid)                                                     # free memory
    ueye.is_ExitCamera(hCam)                                                                        # clean up camera handle
    k+=1
    return k


In [4]:
k=capture_pic(exposure_arr,gain_arr,k)

0
1
2
3
4
5
6
0
FPS: 14.999610010139737
Exposure: 0.9962300000000002
0.5441894648483104
1
FPS: 14.99963250900353
Exposure: 0.9962300000000002
0.061317654562416
2
FPS: 14.999610010139737
Exposure: 0.9962300000000002
0.061930534198647025
3
FPS: 14.999610010139737
Exposure: 0.9962300000000002
0.0622761953810429
4
FPS: 14.999610010139737
Exposure: 0.9962300000000002
0.06257200510977401
5
FPS: 14.99963250900353
Exposure: 0.9962300000000002
0.06076452335962146
6
FPS: 14.999610010139737
Exposure: 0.9962300000000002
0.06526728260092629
7
FPS: 14.999610010139737
Exposure: 0.9962300000000002
0.06677565562909049
8
FPS: 14.999610010139737
Exposure: 4.9970408108108115
0.06207898889522201
9
FPS: 14.999610010139737
Exposure: 4.9970408108108115
0.05960254462346559
10
FPS: 14.99963250900353
Exposure: 4.9970408108108115
0.08277503981701595
11
FPS: 14.999610010139737
Exposure: 4.9970408108108115
0.0643930493877618
12
FPS: 14.999610010139737
Exposure: 4.9970408108108115
0.06611659008725845
13
FPS: 14.9996

IndexError: index 64 is out of bounds for axis 0 with size 64

In [26]:
gain_arr=np.arange(120,420,40)[::-1]
b=gain_arr
for i in range(len(gain_arr)-1):
    gain_arr=np.append(gain_arr,b)
    
exposure_arr=np.arange(1,33,4)
exposure_arr=np.repeat(exposure_arr,len(exposure_arr))
print(gain_arr)
print(exposure_arr)

[400 360 320 280 240 200 160 120 400 360 320 280 240 200 160 120 400 360
 320 280 240 200 160 120 400 360 320 280 240 200 160 120 400 360 320 280
 240 200 160 120 400 360 320 280 240 200 160 120 400 360 320 280 240 200
 160 120 400 360 320 280 240 200 160 120]
[ 1  1  1  1  1  1  1  1  5  5  5  5  5  5  5  5  9  9  9  9  9  9  9  9
 13 13 13 13 13 13 13 13 17 17 17 17 17 17 17 17 21 21 21 21 21 21 21 21
 25 25 25 25 25 25 25 25 29 29 29 29 29 29 29 29]


In [2]:
#from pyueye import ueye
import ctypes
import time
import numpy as np
import os


gain_arr=np.arange(120,420,60)[::-1]
b=gain_arr
print(b)
for i in range(len(gain_arr)-1):
    gain_arr=np.append(gain_arr,b)
    
exposure_arr=np.arange(3,33,7)
exposure_arr=np.repeat(exposure_arr,len(b))
print(len(exposure_arr))
print(len(gain_arr))
k=1

[360 300 240 180 120]
25
25


In [3]:
exposure_arr

array([ 3,  3,  3,  3,  3, 10, 10, 10, 10, 10, 17, 17, 17, 17, 17, 24, 24,
       24, 24, 24, 31, 31, 31, 31, 31])

In [4]:
gain_arr

array([360, 300, 240, 180, 120, 360, 300, 240, 180, 120, 360, 300, 240,
       180, 120, 360, 300, 240, 180, 120, 360, 300, 240, 180, 120])

In [4]:
def clickPics(exposure_arr,gain_arr,k):
    start=time.time()
    hcam = ueye.HIDS(0)
    pccmem = ueye.c_mem_p()
    memID = ueye.c_int()
    hWnd = ctypes.c_voidp()
    ueye.is_InitCamera(hcam, hWnd)
    ueye.is_SetDisplayMode(hcam, 0)
    sensorinfo = ueye.SENSORINFO()
    ueye.is_GetSensorInfo(hcam, sensorinfo)
    ueye.is_AllocImageMem(hcam, sensorinfo.nMaxWidth, sensorinfo.nMaxHeight,24, pccmem, memID)
    ueye.is_SetImageMem(hcam, pccmem, memID)
    ueye.is_SetDisplayPos(hcam, 100, 100)

    newpath=r'C:/Users/sinayak/Project/bitmapImages4/test'+str(k)+'/'
    if not os.path.exists(newpath):
        os.makedirs(newpath)

    for i in range(len(exposure_arr)):

        CamProp_Exposure = ueye.double()
        CamProp_Exposure.value = exposure_arr[i]     # exposure value in ms ; max=32 if FPS=30
        nRet = ueye.is_Exposure (hcam, ueye.IS_EXPOSURE_CMD_SET_EXPOSURE, CamProp_Exposure, 8)                        

        CamProp_ExposureReal = ueye.double()
        nRet = ueye.is_Exposure (hcam, ueye.IS_EXPOSURE_CMD_GET_EXPOSURE, CamProp_ExposureReal, 8)     

        CamProp_Gamma = ueye.int()
        CamProp_Gamma.value = gain_arr[i]
        ueye.is_Gamma(hcam, ueye.IS_GAMMA_CMD_SET, CamProp_Gamma, ueye.sizeof(CamProp_Gamma)) 
        nret = ueye.is_FreezeVideo(hcam, ueye.IS_WAIT)


        FileParams = ueye.IMAGE_FILE_PARAMS()
        FileParams.pwchFileName =newpath+ str(exposure_arr[i])+"_"+str(gain_arr[i])+".bmp"
        #print(newpath+ str(exposure_arr[i])+"_"+str(gain_arr[i])+".bmp")
        FileParams.nFileType = ueye.IS_IMG_BMP
        FileParams.ppcImageMem = None
        FileParams.pnImageID = None


        nret = ueye.is_ImageFile(hcam, ueye.IS_IMAGE_FILE_CMD_SAVE, FileParams, ueye.sizeof(FileParams))

    end=time.time()
    print(end-start)
    ueye.is_FreeImageMem(hcam, pccmem, memID)
    ueye.is_ExitCamera(hcam)
    k+=1
    

In [5]:
for k in range(101):
    clickPics(exposure_arr,gain_arr,k)
    print(k)

3.935631036758423
0
3.931732416152954
1
4.125730752944946
2
4.056467056274414
3
3.989102602005005
4
4.027781963348389
5
4.222979307174683
6
4.002794981002808
7
3.9161407947540283
8
4.028381824493408
9
4.070759296417236
10
4.106441020965576
11
3.735536575317383
12
4.020486831665039
13
3.797603130340576
14
3.716545820236206
15
3.982205867767334
16
3.753325939178467
17
3.9142417907714844
18
3.871864080429077
19
3.672868490219116
20
3.9411275386810303
21
3.9005491733551025
22
3.6647729873657227
23
3.96101713180542
24
3.899949789047241
25
3.917340040206909
26
3.67476749420166
27
3.8387813568115234
28
3.7951042652130127
29
3.9816064834594727
30
3.8206911087036133
31
4.210486173629761
32
3.6772663593292236
33
3.704051971435547
34
3.4514851570129395
35
2.8932783603668213
36
2.618223190307617
37
3.093473196029663
38
3.2145097255706787
39
3.195730447769165
40


OSError: [WinError 112] There is not enough space on the disk: 'C:/Users/sinayak/Project/bitmapImages4/test41/'

In [74]:
k=clickPics(exposure_arr,gain_arr,k)
print(k)

C:/Users/sinayak/Project/bitmapImages/test101/3_360.bmp
C:/Users/sinayak/Project/bitmapImages/test101/3_300.bmp
C:/Users/sinayak/Project/bitmapImages/test101/3_240.bmp
C:/Users/sinayak/Project/bitmapImages/test101/3_180.bmp
C:/Users/sinayak/Project/bitmapImages/test101/3_120.bmp
C:/Users/sinayak/Project/bitmapImages/test101/10_360.bmp
C:/Users/sinayak/Project/bitmapImages/test101/10_300.bmp
C:/Users/sinayak/Project/bitmapImages/test101/10_240.bmp
C:/Users/sinayak/Project/bitmapImages/test101/10_180.bmp
C:/Users/sinayak/Project/bitmapImages/test101/10_120.bmp
C:/Users/sinayak/Project/bitmapImages/test101/17_360.bmp
C:/Users/sinayak/Project/bitmapImages/test101/17_300.bmp
C:/Users/sinayak/Project/bitmapImages/test101/17_240.bmp
C:/Users/sinayak/Project/bitmapImages/test101/17_180.bmp
C:/Users/sinayak/Project/bitmapImages/test101/17_120.bmp
C:/Users/sinayak/Project/bitmapImages/test101/24_360.bmp
C:/Users/sinayak/Project/bitmapImages/test101/24_300.bmp
C:/Users/sinayak/Project/bitmapImage